# Carregar dados

In [1]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

# df = pd.read_parquet(
#     "./output/silver/rais_TODOS_ANOS_TODAS_COLS_RAIS_VINC_PUB.parquet.gzip")

df = pd.read_parquet(
    "./output/silver/rais_TODOS_ANOS_RAIS_VINC_PUB.parquet.gzip")


qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 47035261



,ano,id_uf_municipio,remuneracao_media,mot_desligamento_id,idade,ocupacao_id,tmh_estabalecimento_id,tpo_vinculo_id,fxa_tmp_emprego_id,sexo_id,escolaridade_id,sigla_uf,sigla_municipio
0,2015,120001.0,2217.14,11,44.0,411010,2.0,10.0,4.0,1.0,2.0,AC,Acrelandia
1,2015,120020.0,973.73,21,31.0,411005,7.0,10.0,4.0,1.0,2.0,AC,Cruzeiro do Sul


In [2]:
# df.to_csv("./temp-ana-full.csv")


In [3]:
df[df["remuneracao_media"] > 0]["ano"].value_counts()

2015    7053588
2016    6707850
2018    6671210
2017    6596203
2019    6592033
2020    6362682
2021    6020303
Name: ano, dtype: int64

In [4]:
# df_sexo = pd.read_csv("./output/bronze/planilhas/dic_sexo.csv")
# df_ordered_sinonimo = pd.read_csv("./output/bronze/planilhas/ordered_sinonimo.csv")

In [5]:
# change column ocupacao_id to int
df["ocupacao_id"] = df["ocupacao_id"].astype(int)

# Juntando tabelas

In [6]:

# df = pd.merge(df, df_sexo, left_on="sexo_id", right_on="chave",
#               how="left")
# df = pd.merge(df, df_ordered_sinonimo, left_on="ocupacao_id",
#               right_on="cbo_2002", how="left")
# df.head(2)

### Alterando remuneração média de NaN para 0

In [7]:
df["remuneracao_media"].fillna(0, inplace=True)


# Gerando ID para Sigla_UF

In [8]:
# Create id for sigla_uf
df["sigla_uf_id"] = df["sigla_uf"].astype("category").cat.codes
df.head(2)


,ano,id_uf_municipio,remuneracao_media,mot_desligamento_id,idade,ocupacao_id,tmh_estabalecimento_id,tpo_vinculo_id,fxa_tmp_emprego_id,sexo_id,escolaridade_id,sigla_uf,sigla_municipio,sigla_uf_id
0,2015,120001.0,2217.14,11,44.0,411010,2.0,10.0,4.0,1.0,2.0,AC,Acrelandia,0
1,2015,120020.0,973.73,21,31.0,411005,7.0,10.0,4.0,1.0,2.0,AC,Cruzeiro do Sul,0


## Gerando Regiões por Sigla

In [9]:
# SEPARATE sigla_uf by regions

# https://pt.wikipedia.org/wiki/Regi%C3%B5es_do_Brasil
# https://pt.wikipedia.org/wiki/Unidades_federativas_do_Brasil
REGIOES = {
    "Norte": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
    "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "Centro-Oeste": ["GO", "MT", "MS", "DF"],
    "Sudeste": ["ES", "MG", "RJ", "SP"],
    "Sul": ["PR", "RS", "SC"]
}


In [10]:
df_regiao = df.copy()

# add col Regioes to df_ana with REGIOES
for regiao, siglas in REGIOES.items():
    df_regiao.loc[df["sigla_uf"].isin(
        siglas), "regiao"] = regiao

df_regiao["regiao_id"] = df_regiao["regiao"].astype("category").cat.codes
df = df_regiao
df.head(2)


,ano,id_uf_municipio,remuneracao_media,mot_desligamento_id,idade,ocupacao_id,tmh_estabalecimento_id,tpo_vinculo_id,fxa_tmp_emprego_id,sexo_id,escolaridade_id,sigla_uf,sigla_municipio,sigla_uf_id,regiao,regiao_id
0,2015,120001.0,2217.14,11,44.0,411010,2.0,10.0,4.0,1.0,2.0,AC,Acrelandia,0,Norte,2
1,2015,120020.0,973.73,21,31.0,411005,7.0,10.0,4.0,1.0,2.0,AC,Cruzeiro do Sul,0,Norte,2


# Tamanho estabelecimento
Tamanho do estabelecimento - empregados ativos em 31/12 (10 categorias)

In [11]:
# Tamanho Estabelecimento
TAMANHO_ESTABELECIMENTO = {
    "ZERO": 1,
    "ATE 4": 2,
    "DE 5 A 9":	3,
    "DE 10 A 19":	4,
    "DE 20 A 49":	5,
    "DE 50 A 99":	6,
    "DE 100 A 249":	7,
    "DE 250 A 499":	8,
    "DE 500 A 999":	9,
    "1000 OU MAIS":	10,
    "IGNORADO": -1
}

# Tipo Vínculo
TIPO_VINCULO = {
    "CLT U/PJ IND":	10,
    "CLT U/PF IND":	15,
    "CLT R/PJ IND":	20,
    "CLT R/PF IND":	25,
    "ESTATUTARIO":	30,
    "ESTAT RGPS":	31,
    "ESTAT N/EFET":	35,
    "AVULSO":	40,
    "TEMPORARIO":	50,
    "APREND CONTR":	55,
    "CLT U/PJ DET":	60,
    "CLT U/PF DET":	65,
    "CLT R/PJ DET":	70,
    "CLT R/PF DET":	75,
    "DIRETOR":	80,
    "CONT PRZ DET":	90,
    "CONT TMP DET":	95,
    "CONT LEI EST":	96,
    "CONT LEI MUN":	97,
    "IGNORADO ": -1,
}

# Faixa Tempo Emprego
FAIXA_TEMPO_EMPREGO = {
    "Ate 2, 9 meses": 1,
    "3,0 a 5,9 meses": 2,
    "6,0 a 11,9 meses": 3,
    "12,0 a 23,9 meses": 4,
    "24,0 a 35,9 meses": 5,
    "36,0 a 59,9 meses": 6,
    "60,0 a 119,9 meses": 7,
    "120,0 meses ou mais": 8
}

# Tipo Salário
# TIPO_SALARIO = {
#     "MENSAL":	1,
#     "QUINZENAL":	2,
#     "SEMANAL":	3,
#     "DIÁRIO":	4,
#     "HORÁRIO":	5,
#     "TAREFA":	6,
#     "OUTROS":	7,
#     "IGNORADO ": -1,
# }

# Motivo Desligamento
MOTIVO_DESLIGAMENTO = {
    "DEM COM JC":	10, # Rescisão com justa causa por iniciativa do empregador ou servidor demitido
    "DEM SEM JC":	11, # Rescisão sem justa causa por iniciativa do empregador.
    "TERM CONTR":	12, # Término do contrato de trabalho.
    "DESL COM JC":	20, # Rescisão com justa causa por iniciativa do empregado(rescisão indireta).
    "DESL SEM JC":	21, # Rescisão sem justa causa por iniciativa do empregado ou exoneração a pedido
    "POSS OUT CAR":	22, # Posse em outro cargo inacumulável(específico para servidor público)
    "TRANS C/ONUS":	30, # Transferência de empregado entre estabelecimentos da mesma empresa ou para outra empresa, com ônus para a cedente
    "TRANS S/ONUS":	31, # Transferência de empregado entre estabelecimentos da mesma empresa ou para outra empresa, sem ônus para a cedente
    "READAP/REDIS":	32, # Readaptação ou redistribuição(específico para servidor publico)
    "CESSAO":	33, # Cessão
    "REDISTRIBUIÇÃO":	34, # Redistribuição(específico para servidor publico)
    "MUD. REGIME":	40, # Mudança de regime trabalhista.
    "REFORMA":	50, # Reforma de militar para a reserva remunerada.
    "FALECIMENTO":	60, # Falecimento.
    "FALEC AC TRB":	62, # Falecimento decorrente de acidente do trabalho típico(que ocorre no exercício de atividades profissionais a serviço da empresa)
    "FALEC AC TIP":	63, # Falecimento decorrente de acidente do trabalho de trajeto(ocorrido no trajeto residência-trabalho-residência)
    "FALEC D PROF":	64, # Falecimento decorrente de doença profissional.
    "APOS TS CRES":	70, # Aposentadoria por tempo de serviço, com rescisão contratual.
    "APOS TS SRES":	71, # Aposentadoria por tempo de serviço, sem rescisão contratual.
    "APOS ID CRES":	72, # Aposentadoria por idade, com rescisão contratual.
    "APOS IN ACID":	73, # Aposentadoria por invalidez, decorrente de acidente do trabalho.
    "APOS IN DOEN":	74, # Aposentadoria por invalidez, decorrente de doença profissional.
    "APOS COMPULS":	75, # Aposentadoria compulsória.
    "APOS IN OUTR":	76, # Aposentadoria por invalidez, exceto a decorrente de doença profissional ou acidente do trabalho.
    "APOS ID SRES":	78, # Aposentadoria por idade, sem rescisão contratual.
    "APOS ESP CRE":	79, # Aposentadoria especial, com rescisão contratual.
    "APOS ESP SRE":	80, # Aposentadoria especial, sem rescisão contratual.
    "DESL POR ACORDO":	90, # Desligamento por Acordo Empregado e Empregador
    "NAO DESL ANO":	0, # Não desligado no ano
    "IGNORADO": -1
}

# Sexo Trabalhador
SEXO = {
    "Masculino": 1,
    "Feminino": 2,
    "IGNORADO": -1
}

# CBO Ocupação 2002
OCUPACAO = {
    "Engenheiro de Aplicativos em Computacao": 212205,
    "Engenheiro de Equipamentos em Computacao": 212210,
    "Engenheiros de Sistemas Operacionais em Computacao": 212215,
    "Administrador de Banco de Dados": 212305,
    "Administrador de Redes": 212310,
    "Administrador de Sistemas Operacionais": 212315,
    "Administrador em Segurança da Informação": 212320,
    "Analista de Desenvolvimento de Sistemas": 212405,
    "Analista de Redes e de Comunicacao de Dados": 212410,
    "Analista de Sistemas de Automacao": 212415,
    "Analista de Suporte Computacional": 212420,
    "Programador de Internet": 317105,
    "Programador de Sistemas de Informacao": 317110,
    "Programador de Maquinas - Ferramenta com Comando Numerico": 317115,
    "Programador de Multimidia": 317120,
    "Operador de Computador(Inclusive Microcomputador)": 317205,
    "Tecnico de Apoio ao Usuario de Informatica(Helpdesk)": 317210,
}

#  Escolaridade após 2005
ESCOLARIDADE = {
    "ANALFABETO":	1,
    "ATE 5.A INC": 2,
    "5.A CO FUND": 3,
    "6. A 9. FUND": 4,
    "FUND COMPL":	5,
    "MEDIO INCOMP":	6,
    "MEDIO COMPL":	7,
    "SUP. INCOMP":	8,
    "SUP. COMP":	9,
    "MESTRADO": 10,
    "DOUTORADO": 11,
    "IGNORADO": -1,
}

In [12]:
# COLS_RENAME = {
#     "Tamanho Estabelecimento": "tmh_estabalecimento_id",
#     "Tipo Vínculo": "tpo_vinculo_id",
#     "Faixa Tempo Emprego": "fxa_tmp_emprego_id",
#     "Tipo Salário": "tpo_salario_id",
#     "desligamento_id": "mot_desligamento_id",
#     "Sexo Trabalhador": "sexo_id",
#     "Escolaridade após 2005": "escolaridade_id"
# }
# df_renom = df.rename(columns=COLS_RENAME).copy()
# df_renom.head(2)

df_renom = df.copy()


In [13]:
COLS_VALUES = {
    "tmh_estabalecimento_id":TAMANHO_ESTABELECIMENTO,
    "tpo_vinculo_id":TIPO_VINCULO,
    "fxa_tmp_emprego_id":FAIXA_TEMPO_EMPREGO,
    # "tpo_salario_id":TIPO_SALARIO,
    "mot_desligamento_id":MOTIVO_DESLIGAMENTO,
    "sexo_id":SEXO,
    "ocupacao_id":OCUPACAO,
    "escolaridade_id":ESCOLARIDADE
}

def criar_cols_com_valores(df):
  df_d = df.copy()

  for nome_col_id, obj_k_v in COLS_VALUES.items():
    nome_col_sem_id = nome_col_id.replace("_id", "")
    # df_d[nome_col_id].replace("-1", 9999, inplace=True)
    df_d[nome_col_id].fillna(-1, inplace=True)
    df_d[nome_col_id].astype(int)
    df_d[nome_col_sem_id] = "NA"
    # print("1== ", nome_col_id, nome_col_sem_id, obj_k_v)
    for sub_k, sub_v in obj_k_v.items():
      # print("2== ", sub_k, sub_v)
      df_d.loc[df_d[nome_col_id] == sub_v, nome_col_sem_id] = sub_k

  return df_d


df_n = criar_cols_com_valores(df_renom)
df_n.head(2)


,ano,id_uf_municipio,remuneracao_media,mot_desligamento_id,idade,ocupacao_id,tmh_estabalecimento_id,tpo_vinculo_id,fxa_tmp_emprego_id,sexo_id,escolaridade_id,sigla_uf,sigla_municipio,sigla_uf_id,regiao,regiao_id,tmh_estabalecimento,tpo_vinculo,fxa_tmp_emprego,mot_desligamento,sexo,ocupacao,escolaridade
0,2015,120001.0,2217.14,11,44.0,411010,2.0,10.0,4.0,1.0,2.0,AC,Acrelandia,0,Norte,2,ATE 4,CLT U/PJ IND,"12,0 a 23,9 meses",DEM SEM JC,Masculino,NA,ATE 5.A INC
1,2015,120020.0,973.73,21,31.0,411005,7.0,10.0,4.0,1.0,2.0,AC,Cruzeiro do Sul,0,Norte,2,DE 100 A 249,CLT U/PJ IND,"12,0 a 23,9 meses",DESL SEM JC,Masculino,NA,ATE 5.A INC


# Selecionando colunas necessárias

In [14]:
df_n = df_n[["ano", "regiao_id", "regiao", "sigla_uf_id", "sigla_uf", "sigla_municipio", "remuneracao_media", "idade",
           "tmh_estabalecimento_id", "tmh_estabalecimento",
           "tpo_vinculo_id", "tpo_vinculo",
           "fxa_tmp_emprego_id", "fxa_tmp_emprego",
          #  "tpo_salario_id", "tpo_salario",
           "mot_desligamento_id", "mot_desligamento",
           "sexo_id", "sexo",
           "ocupacao_id", "ocupacao",
           "escolaridade_id", "escolaridade",
         ]]

df_n.head(2)


,ano,regiao_id,regiao,sigla_uf_id,sigla_uf,sigla_municipio,remuneracao_media,idade,tmh_estabalecimento_id,tmh_estabalecimento,tpo_vinculo_id,tpo_vinculo,fxa_tmp_emprego_id,fxa_tmp_emprego,mot_desligamento_id,mot_desligamento,sexo_id,sexo,ocupacao_id,ocupacao,escolaridade_id,escolaridade
0,2015,2,Norte,0,AC,Acrelandia,2217.14,44.0,2.0,ATE 4,10.0,CLT U/PJ IND,4.0,"12,0 a 23,9 meses",11,DEM SEM JC,1.0,Masculino,411010,NA,2.0,ATE 5.A INC
1,2015,2,Norte,0,AC,Cruzeiro do Sul,973.73,31.0,7.0,DE 100 A 249,10.0,CLT U/PJ IND,4.0,"12,0 a 23,9 meses",21,DESL SEM JC,1.0,Masculino,411005,NA,2.0,ATE 5.A INC


# Verificando se tem mais algum valor Nulo

In [15]:
df_n[df_n.isna().any(axis=1)]

,ano,regiao_id,regiao,sigla_uf_id,sigla_uf,sigla_municipio,remuneracao_media,idade,tmh_estabalecimento_id,tmh_estabalecimento,tpo_vinculo_id,tpo_vinculo,fxa_tmp_emprego_id,fxa_tmp_emprego,mot_desligamento_id,mot_desligamento,sexo_id,sexo,ocupacao_id,ocupacao,escolaridade_id,escolaridade


# Salvando arquivo

In [ ]:
# df_n["country"] = "Brazil"

In [16]:
df_n.to_parquet("./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip",
                index=False, compression="gzip")


# Filtrando arquivos

In [1]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet(
    "./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip")


qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 47035261



,ano,regiao_id,regiao,sigla_uf_id,sigla_uf,sigla_municipio,remuneracao_media,idade,tmh_estabalecimento_id,tmh_estabalecimento,tpo_vinculo_id,tpo_vinculo,fxa_tmp_emprego_id,fxa_tmp_emprego,mot_desligamento_id,mot_desligamento,sexo_id,sexo,ocupacao_id,ocupacao,escolaridade_id,escolaridade
0,2015,2,Norte,0,AC,Acrelandia,2217.14,44.0,2.0,ATE 4,10.0,CLT U/PJ IND,4.0,"12,0 a 23,9 meses",11,DEM SEM JC,1.0,Masculino,411010,NA,2.0,ATE 5.A INC
1,2015,2,Norte,0,AC,Cruzeiro do Sul,973.73,31.0,7.0,DE 100 A 249,10.0,CLT U/PJ IND,4.0,"12,0 a 23,9 meses",21,DESL SEM JC,1.0,Masculino,411005,NA,2.0,ATE 5.A INC


In [14]:
IDS_PROFISSIONAIS_TI = ["212205", "212210", "212215", "212305", "212310", "212315", "212320", "212405", "212410", "212415", "212420", "317105", "317110", "317115", "317120", "317205", "317210"]
OUTROS_PROFISSIONAIS = [
    "241005", # Advogado
    "252105", # Administrador
    "252210", # Contador
    "411010", # Assistente Administrativo
    "411005", # Auxiliar de Escritorio, em Geral
]

# IDS_PROFISSIONAIS_TI to int
IDS_PROFISSIONAIS_TI = [int(x) for x in IDS_PROFISSIONAIS_TI]
OUTROS_PROFISSIONAIS = [int(x) for x in OUTROS_PROFISSIONAIS]

In [15]:
df[df['ocupacao_id'].isin(IDS_PROFISSIONAIS_TI)].to_csv("./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.csv",
              decimal=",", sep=";", index=False)

In [16]:
df[df['ocupacao_id'].isin(OUTROS_PROFISSIONAIS)].to_csv("./output/gold/OUTROS_2018_2019_rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.csv",
              decimal=",", sep=";", index=False)